Create a file with data from closest grid point
===

Import required modules and set file names
----

In [1]:
from netCDF4 import Dataset,date2num
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2
from model import Model
from GCNet import GCNet

In [2]:
# modelDataFN = "daily/ei.oper.an.sfc.regn128sc.tas.19960101-20141231.nc"
suff = "19960101-20171231"
modelDataFN = "daily/ei.oper.an.sfc.regn128sc.tas."+suff+"_sub.nc"
modelMetaFN = "erai_geog_sub.nc"
newNCFN = "erai_tas_closest_"+suff+".nc"

Load/define ERA Interim metadata
----

In [3]:
M = Model(modelMetaFN)
M.loadMeta()

minLon = -70 + 360
maxLon = -15 + 360
minLat = 58
maxLat = 86.5
M.setDomain( (minLat, maxLat, minLon, maxLon) )

1d dataset
1 40 78 3120


In [5]:
g = M.getMeta()
g[-1]

(39, 58.59631348, 77, 344.53137207, 0.)

Load AWS metadata
----

In [6]:
A = GCNet("site_info.nc")
A.loadMeta()
nSites = A.getNSites()

Find closest grid points
-----

In [7]:
C = []
# for awsIX in (range(nSites)):
# for awsIX in ((0,1)):
for awsIX in ((0,)):
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))
    closest = M.closestPoints( awsLatLon )
    C.append( closest )
    if awsIX == 5:
        print closest.dtype
        print closest

[(23, 69.82426452636719, 29, 310.7813720703125, 28.081253544095343)]
2 [(23, 69.82426452636719, 29, 310.7813720703125, 28.081253544095343), (23, 69.82426452636719, 28, 310.0782470703125, 36.93825353732116)]
3 [(23, 69.82426452636719, 29, 310.7813720703125, 28.081253544095343), (23, 69.82426452636719, 28, 310.0782470703125, 36.93825353732116)]
4 [(23, 69.82426452636719, 29, 310.7813720703125, 28.081253544095343), (23, 69.82426452636719, 28, 310.0782470703125, 36.93825353732116), (24, 69.12252044677734, 29, 310.7813720703125, 50.216902020173784)]
5 [(23, 69.82426452636719, 29, 310.7813720703125, 28.081253544095343), (23, 69.82426452636719, 28, 310.0782470703125, 36.93825353732116), (24, 69.12252044677734, 29, 310.7813720703125, 50.216902020173784), (24, 69.12252044677734, 28, 310.0782470703125, 55.82804612178336)]


Process tas data
----

In [8]:
print modelDataFN
D = Model(None, modelDataFN)

time, timeCF = D.loadData( "time" )
print time[0], time[-1]
# dfTime = pd.Series( time, name="Time")

tas = np.array( D.loadData( "tas" ) )
nRec, nLat, nLon = tas.shape
print nRec, nLat, nLon

Tnew = np.empty( (nRec, nSites), dtype = 'float' )

for awsIX in (range(nSites)):
# for awsIX in ((0,1)):
# for awsIX in ([5]):
    """ Basic AWS info """
    awsName = A.getName( awsIX )
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))

    closest = C[awsIX][0]
#     print awsLatLon, closest
#     print closest['lat'][0],closest['lon'][0]
    dx = "%.1f km" % closest[-1]
    print awsName, dx
    Tnew[:,awsIX] = tas[:,closest['ixLat'],closest['ixLon']]

daily/ei.oper.an.sfc.regn128sc.tas.19960101-20171231_sub.nc
1996-01-01 09:00:00 2017-12-31 09:00:00
8030 40 78
Swiss Camp 28.1 km
Crawford Point1 8.3 km
NASA-U 24.0 km
GITS 33.6 km
Humboldt 31.4 km
Summit 8.0 km
Tunu-N 26.0 km
DYE-2 19.2 km
JAR1 37.4 km
Saddle 36.2 km
South Dome 39.2 km
NASA-E 30.1 km
Crawford Point2 13.9 km
NGRIP 37.9 km
NASA-SE 23.0 km
KAR 14.0 km
JAR2 33.5 km
KULU 19.1 km
JAR3 32.8 km
Aurora 15.3 km
Petermann GL 37.2 km
Petermann ELA 30.2 km
NEEM 7.5 km


In [7]:
Tnew[0:10,5]

array([-25.54381409, -31.43120422, -29.59216919, -28.76265564,
       -30.25147095, -32.19724121, -30.22975769, -29.13880005,
       -32.39005127, -33.59685364])

Create a netCDF file for the new data
-----

In [9]:
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass

ncfile = Dataset(newNCFN,mode='w',format='NETCDF4_CLASSIC') 
ncfile.title = "Closest-point temperature data"
ncfile.period = suff

time_dim = ncfile.createDimension('time', None)
# ens_dim = ncfile.createDimension('ensemble', nEns)
stn_dim = ncfile.createDimension('station', nSites)

timeVar = ncfile.createVariable('time', np.float32, ('time',))
timeVar.units = timeCF.units
timeVar.calendar = timeCF.calendar
timeVar.standard_name = timeCF.standard_name
timeVar.long_name = timeCF.long_name
timeVar.axis = timeCF.axis

timeVar[:] = date2num(time, timeCF.units, timeCF.calendar)

# tasVar = ncfile.createVariable('tas', np.float32, ('time','station'))
# tasVar = ncfile.createVariable('tas', np.float32, ('ensemble','time','station'))
tasVar = ncfile.createVariable('tas', np.float32, ('time','station'))
tasVar.units = "K"
tasVar.description = "Surface temperature, closest point"
tasVar.long_name = "Reference height temperature"
# tasVar[:,:] = Tnew
tasVar[:] = Tnew

ncfile.close()